In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import pprint
import re
import numpy as np
import math

In [2]:
df = pd.read_pickle("./data/article_contents_20211010.pkl")

In [3]:
df.head(30)  

,title,mdName,content,url
0,"대구보건대 안경광학과, 현장직무수행능력 평가 실시",헤럴드경제,지난 6일 열린 대구보건대 안경광학과 현장직무수행능력 평가 모습.[대구보건대 제공]...,https://news.naver.com/main/read.naver?mode=LS...
1,ITALY SOCCER UEFA NATIONS LEAGUE,EPA연합뉴스,Italy vs BelgiumDomenico Berardi (bottom) of I...,https://news.naver.com/main/read.naver?mode=LS...
2,EPA OUTLOOK,EPA연합뉴스,EPA OUTLOOKWe expect to move pictures on the f...,https://news.naver.com/main/read.naver?mode=LS...
3,ITALY SOCCER UEFA NATIONS LEAGUE,EPA연합뉴스,Italy vs BelgiumKevin De Bruyne (C) of Belgium...,https://news.naver.com/main/read.naver?mode=LS...
4,"경북도, 한글비전 선포식 개최",헤럴드경제,경북도 한글비전 선포식 개최 모습.[경북도 제공] [헤럴드경제(안동)=김병진 기자]...,https://news.naver.com/main/read.naver?mode=LS...
5,"이낙연 측, ""무효표 이의제기""...경선 후유증 현실화되나",YTN,동영상 뉴스\n\n\n\n\n\t\n\t[앵커]민주당 대선 후보로 이재명 경기지사가...,https://news.naver.com/main/read.naver?mode=LS...
6,Turkey F1 GP Auto Racing,AP연합뉴스,Mercedes driver Valtteri Bottas of Finland ste...,https://news.naver.com/main/read.naver?mode=LS...
7,예상 뒤엎은 아슬아슬 과반...결국 대장동 의혹 영향,YTN,동영상 뉴스\n\n\n\n\n\t\n\t[앵커]이재명 후보는 최종 누적 득표율 50...,https://news.naver.com/main/read.naver?mode=LS...
8,"롯데백화점 대구점, 가을맞이 라이프 스타일 홈페어 개최",헤럴드경제,[롯데백화점 제공][헤럴드경제(대구)=김병진 기자]사회적 거리두기 강화 방침이 계속...,https://news.naver.com/main/read.naver?mode=LS...
9,Italy Belgium Nations League Soccer,AP연합뉴스,"Belgium's Charles De Ketelaere, left, celebrat...",https://news.naver.com/main/read.naver?mode=LS...


In [4]:
vocab_dict = {}
idx = 0
for i, row in df.iterrows():
    title = row['title']
    
    terms = title.split()
    for term in terms:
        if term not in vocab_dict:
            vocab_dict[term] = idx
            idx+=1
vocab_dict

{'대구보건대': 0,
 '안경광학과,': 1,
 '현장직무수행능력': 2,
 '평가': 3,
 '실시': 4,
 'ITALY': 5,
 'SOCCER': 6,
 'UEFA': 7,
 'NATIONS': 8,
 'LEAGUE': 9,
 'EPA': 10,
 'OUTLOOK': 11,
 '경북도,': 12,
 '한글비전': 13,
 '선포식': 14,
 '개최': 15,
 '이낙연': 16,
 '측,': 17,
 '"무효표': 18,
 '이의제기"...경선': 19,
 '후유증': 20,
 '현실화되나': 21,
 'Turkey': 22,
 'F1': 23,
 'GP': 24,
 'Auto': 25,
 'Racing': 26,
 '예상': 27,
 '뒤엎은': 28,
 '아슬아슬': 29,
 '과반...결국': 30,
 '대장동': 31,
 '의혹': 32,
 '영향': 33,
 '롯데백화점': 34,
 '대구점,': 35,
 '가을맞이': 36,
 '라이프': 37,
 '스타일': 38,
 '홈페어': 39,
 'Italy': 40,
 'Belgium': 41,
 'Nations': 42,
 'League': 43,
 'Soccer': 44,
 '대구도시공사,': 45,
 'ESG금융인증': 46,
 '최고등급': 47,
 '획득': 48,
 'SPAIN': 49,
 'TRIATHLON': 50,
 'Israel': 51,
 'Palestinians': 52,
 '도지사': 53,
 '직인': 54,
 '등': 55,
 '155개': 56,
 '공인': 57,
 '훈민정음체로': 58,
 '교체': 59,
 'NORWAY': 60,
 'WRESTLING': 61,
 'WORLD': 62,
 'CHAMPIONSHIPS': 63,
 '홍준표,': 64,
 '옹호…“사퇴': 65,
 '후보표': 66,
 '무효?': 67,
 '민주당': 68,
 '해석': 69,
 '어이없어”': 70,
 '대선': 71,
 '후보로': 72,
 '이재명': 73,
 '선출...본

In [5]:
len(vocab_dict)

20274

In [6]:
def title2idx(title, vocab_dict):
    result = []
    terms = set(title.split())
    for term in terms:
        result.append(vocab_dict[term])
    return result

In [7]:
idx_list = []
for i, row in df.iterrows():
    idxs = title2idx(row['title'], vocab_dict)
    idx_list.append(idxs)
idx_list

[[2, 1, 3, 4, 0],
 [6, 9, 8, 7, 5],
 [10, 11],
 [6, 9, 8, 7, 5],
 [12, 13, 14, 15],
 [17, 21, 20, 16, 19, 18],
 [24, 26, 25, 22, 23],
 [31, 28, 32, 33, 29, 30, 27],
 [37, 39, 35, 38, 36, 34, 15],
 [42, 44, 43, 41, 40],
 [47, 48, 46, 45],
 [6, 9, 8, 7, 5],
 [50, 49],
 [51, 52],
 [6, 9, 8, 7, 5],
 [53, 55, 56, 12, 54, 59, 58, 57],
 [6, 9, 8, 7, 5],
 [63, 60, 62, 61],
 [66, 70, 65, 67, 16, 68, 69, 64],
 [42, 44, 43, 41, 40],
 [63, 60, 62, 61],
 [63, 60, 62, 61],
 [6, 9, 8, 7, 5],
 [72, 71, 75, 68, 74, 73],
 [77, 80, 79, 78, 76],
 [77, 80, 79, 78, 76],
 [6, 9, 8, 7, 5],
 [63, 60, 62, 61],
 [6, 9, 8, 7, 5],
 [6, 9, 8, 7, 5],
 [77, 80, 79, 78, 76],
 [6, 9, 8, 7, 5],
 [6, 9, 8, 7, 5],
 [6, 9, 8, 7, 5],
 [50, 49],
 [77, 80, 79, 78, 76],
 [50, 49],
 [42, 44, 43, 41, 40],
 [50, 49],
 [77, 80, 79, 78, 76],
 [77, 80, 79, 78, 76],
 [77, 80, 79, 78, 76],
 [50, 49],
 [50, 49],
 [50, 49],
 [50, 49],
 [50, 49],
 [85, 55, 86, 82, 84, 83, 81],
 [92, 89, 91, 88, 87, 90, 93],
 [95, 94, 96, 97],
 [50, 49],


In [8]:
sim_dict = {}
already = set()
for i in tqdm(range(len(idx_list))):
    if i in already:
        continue
        
    for j in range(i+1,len(idx_list)):
        stand_len = len(idx_list[i]) if len(idx_list[i]) < len(idx_list[j]) else len(idx_list[j])
        inter_len = len(set(idx_list[i]) & set(idx_list[j]))
        if (inter_len / stand_len) >= 0.5:
            if i in sim_dict:
                sim_dict[i].append(j)
            else:
                sim_dict[i] = [j]
            already.add(j)
                
sim_dict

{1: [3,
  11,
  14,
  16,
  22,
  26,
  28,
  29,
  31,
  32,
  33,
  87,
  88,
  93,
  94,
  95,
  96,
  99,
  100,
  101,
  102,
  109,
  110,
  112,
  116,
  128,
  141,
  151,
  153,
  167,
  169,
  177,
  180,
  182,
  184,
  196,
  198,
  202,
  205,
  219,
  222,
  225,
  226,
  227,
  229,
  239,
  241,
  242,
  243,
  245,
  246,
  247,
  249,
  250,
  262,
  263,
  277,
  285,
  296,
  301,
  302,
  316,
  320,
  340,
  341,
  342,
  343,
  363,
  365,
  371,
  373,
  386,
  391,
  408,
  422,
  425,
  435,
  448,
  452,
  455,
  460,
  466,
  488,
  490,
  493,
  539,
  540,
  548,
  566,
  567,
  570,
  578,
  580,
  582,
  583,
  645,
  656,
  668,
  682,
  721,
  726,
  745,
  759,
  765,
  777,
  778,
  813,
  823,
  852,
  855,
  857,
  911,
  967,
  7539,
  7579,
  7580,
  7581,
  7583,
  7584,
  8960,
  9207,
  9208,
  9211,
  9212,
  9216,
  9217,
  9218,
  9219,
  9220],
 2: [3567, 9513],
 4: [3273, 5052],
 5: [1831, 1983, 3340, 3471],
 6: [83, 91, 92, 278, 412, 413

In [9]:
for stand_idx in sim_dict:
    print(df['title'][stand_idx], stand_idx)
    for idx in sim_dict[stand_idx]:
        print(df['title'][idx])
    print("\n")

ITALY SOCCER UEFA NATIONS LEAGUE 1
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SOCCER UEFA NATIONS LEAGUE
ITALY SO

5대 은행 대출 증가율 이미 5%…연쇄 '대출 중단' 현실화되나
5대 은행 대출 증가율 이미 5%…연쇄 '대출 중단' 임박
5대 은행 대출 증가율 이미 5%에 육박
5대 은행 대출 증가율 이미 5%…연쇄 '대출 중단' 임박
5대 은행 대출 증가율 이미 5%…연쇄 '대출 중단' 임박


악수하는 이재명-이낙연 418
이재명 후보와 악수하는 이낙연 후보
악수하는 이낙연-이재명 후보
꽃다발 들고 마주보는 이재명-이낙연
송영길 대표와 악수하는 이재명 후보
송영길 대표와 악수하는 이재명 대선 후보
후보 확정 후 이낙연과 악수하는 이재명
두 손 맞잡은 이재명-이낙연
자리로 향하는 이재명-이낙연 후보
이동하는 이재명-이낙연 후보
기념촬영 마친 민주당 이재명-이낙연 대선 경선 후보
악수하는 이재명-이낙연
악수 나누는 이재명-이낙연 후보
생각에 잠긴 민주당 이재명-이낙연 대선 경선 후보
정견 발표 앞둔 민주당 이재명-이낙연 대선 경선 후보
악수하는 이낙연과 이재명
포토타임 마치고 이동하는 이재명-이낙연 후보


이재명 첫 일정 대전현충원…박정희 이승만 참배 문제 피해갈듯 433
기념촬영하는 이재명
기뻐하는 이재명
기뻐하는 이재명
인사하는 이재명
연설하는 이재명
연설하는 이재명


NETHERLANDS SOCCER FIFA WORLD CUP 2022 QUALIFYING 439
NETHERLANDS SOCCER FIFA WORLD CUP 2022 QUALIFYING
NETHERLANDS SOCCER FIFA WORLD CUP 2022 QUALIFYING
NETHERLANDS SOCCER FIFA WORLD CUP 2022 QUALIFYING
SWITZERLAND SOCCER FIFA WORLD CUP 2022 QUALIFIER
SWITZERLAND SOCCER FIFA WORLD CUP 2022 QUALIFIER
SWITZERLAND SOCCER FIFA WORLD CUP 2022 QUALIFIER
SWITZERLAND SOCCER FIFA WORLD CUP 2022 QUALIF

적재 '별 보러 가자' 1118
적재 '별 보러 가자'


이재명 "문제는 리더…이번 대선, 부패 기득권과의 최후대첩" [전문] 1120
기념촬영하는 이재명
기뻐하는 이재명
기뻐하는 이재명
인사하는 이재명
연설하는 이재명
이재명 "대선, 부패 기득권 세력과의 최후대첩"
연설하는 이재명


적재의 열띤 공연 1121
적재의 열띤 공연
더 발룬티어스의 열띤 공연
더 발룬티어스의 열띤 공연
열띤 공연 펼치는 엔플라잉
열띤 공연 펼치는 엔플라잉
열띤 공연 펼치는 엔플라잉
열띤 공연 펼치는 엔플라잉
김사월의 열띤 공연
김사월의 열띤 공연
김사월의 열띤 공연
열띤 공연 펼치는 실리카겔
열띤 공연 펼치는 실리카겔
열띤 공연 펼치는 실리카겔
헤이맨의 열띤 공연
헤이맨의 열띤 공연
열띤 공연 선보이는 헤이맨
열띤 공연 선보이는 헤이맨
열띤 공연 선보이는 헤이맨


더 발룬티어스의 비대면 록 페스티벌 1125
엔플라잉의 비대면 락 페스티벌
비대면 인천 펜타포트 록 페스티벌


더 발룬티어스 백예린 '열창' 1126
더 발룬티어스 백예린 '열창'
더 발룬티어스 백예린 '열창'
엔플라잉 '열창'
엔플라잉 '열창'
엔플라잉 '열창'


불꽃놀이·야회로 당창건 기념하는 북한 주민들 1129
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들
불꽃놀이·야회로 당창건 기념하는 북한 주민들


이낙연 측 사실상 경선불복…‘무효표 처리’ 이의제기키로 1142
[속보] 이낙


윤석열 "홍 선배님, 우리 깐부 아닌가요?"… 홍준표 "이상한 짓 말자" 1970
윤석열 “홍 선배님, 우리 깐부 아닌가요” 홍준표 “동지끼리는 음해 안 해”
홍준표 압승하겠습니다
기도하는 윤석열
윤석열 "홍 선배님, 우리 깐부 아닌가요" 홍준표 "동지는 음해 안해"
기도하는 윤석열


민주당 경선 마치고 나서는 이낙연 1972
퇴장하는 이낙연
민주당 대선 경선 '피날레'
경선 마치고 인사하는 이재명・이낙연
민주당 대선 경선 '피날레'
잠시 생각에 잠긴 민주당 이낙연 경선 후보
정견 발표하는 민주당 이낙연 대선 경선 후보
정견 발표하는 민주당 이낙연 대선 경선 후보
정견 발표하는 민주당 이낙연 대선 경선 후보
박수치는 민주당
연설하는 이낙연
목타는 이낙연
정견 발표하는 민주당 이낙연 대선 경선 후보
정견 발표하는 민주당 이낙연 대선 경선 후보
정견 발표하는 민주당 이낙연 대선 경선 후보
정견 발표하는 민주당 이낙연 대선 경선 후보
정견 발표하는 민주당 이낙연 대선 경선 후보
정견 발표하는 민주당 이낙연 대선 경선 후보
마지막 경선 연설 이낙연 “대장동 비리, 민주당 앞길 가로막아”


꽃목걸이 선물 받는 이낙연 1979
퇴장하는 이낙연
이낙연 축하 받는 이재명
꽃목걸이 받은 이낙연 전 대표
꽃목걸이 받는 이낙연
지지자들에게 꽃목걸이 받는 이낙연 전 대표
이낙연 축하 받는 이재명
연설하는 이낙연
목타는 이낙연


한은 "유로지역 GDP, 올해 4분기 코로나 이전 수준 회복" 1980
한은 "유로 경제, 4분기엔 코로나19 이전 수준 회복"
한은 “유로 GDP 4분기 중 코로나 이전 수준 회복…세계경기회복 기여 전망”
한은 "유로지역 GDP, 올 4분기 코로나 이전 수준 회복"


이재명 '엄지 척' 2020
인사하는 이재명
연설하는 이재명
연설하는 이재명
이재명 '엄지 척'


결선 실패 이낙연 "책임 있는 마음으로 기다려달라" 2032
연설하는 이낙연
목타는 이낙연


'선수들에게 박수 보내는 전창진 감독' 2033
'심판에게 항의하는 전창진 감독'


'슛

‘플루트가 만난 낭만’…광주문화재단 월요콘서트 3604
'낭만을 만난 플루트'…그레이스플루트 앙상블 월요콘서트


"결혼 절대 안해, 최악…끔찍한 소리 하지마라" 유부녀 이다영, 1년전 방송서 발언 '재조명' 3605
2018년 결혼한 이다영, 1년전 방송서 "결혼 절대 안해, 끔찍"


'위드 코로나' 한달 앞으로…이번주 민관 일상회복위 출범 3611
'위드코로나' 언제쯤…이번주 일상회복위 출범


결연한 표정의 민주당 대선주자들 3619
포즈 취하는 민주당 대선주자들
입장하는 민주당 대선주자들
손 들어 인사하는 민주당 대선주자들
마지막 무대 오른 민주당 대선주자들
마지막 무대 오르는 민주당 대선주자들


입장하는 송영길 대표와 당 지도부 3622
입장하는 후보들
합동연설회 입장하는 민주당 지도부
서울 합동연설회 참석하는 민주당 송영길 대표와 지도부


ROTC 지원 장려금 400만원→600만원 추진…복무기간 단축 검토 3626
학사·ROTC 장려금 50% 상향…복무기간 단축 검토


성경 들고 교회 찾은 윤석열, 1시간 예배…“천주교·불교도 믿어” 3628
‘무속논란’ 윤석열, 성경 들고 예배…기도에 찬송가도


[내일날씨] 전국 흐리고 비…낮 기온 20도 내외 쌀쌀 3632
전국 흐리고 곳곳 비…낮 최고 29도


윤석열, 성경책 들고 교회 찾고 여름성경학교 사진도…‘무속논란’ 돌파? 3637
윤석열, 성경책 들고 교회 예배 참석…'王'자 무속논란 돌파?
성경책 들고 교회 가는  윤석열 후보
[포토]'무속 논란' 윤석열, 성경책 들고 교회로


"文정부 5년간 서울 주택 소유자 재산세 2배 증가" 3641
서울 주택 재산세 5년새 두배 뛰었다…서초·송파·성동순
서울 주택 재산세 5년새 2배↑···강남·서초·송파 순


與, '王'자 윤석열 예배에 "하나님 이름 욕되게 말라" 3642
기도하는 윤석열
기도하는 윤석열


제주 관광지마다 북적북적 3643
제주 관광지마다 북적북적
제주 관광지마다 북적북적
한글날 연휴, 제주 관광지마다 북적북적
한글날 연휴, 


'묵념' 5296
청년 일자리 총파업 선포 기자회견에 앞서 '묵념'


박찬욱 '금자씨로 보는 광기의 형상' 5302
'금자씨로 보는 광기의 형상'


영화 '친절한 금자씨' 통해 보는 광기의 형상 5303
'친절한 금자씨'를 통해 보는 광기의 형상
'금자씨로 보는 광기의 형상'


포항서 인근 상가로 돌진한 택시… 1명 부상 5311
경북 포항서 매장으로 돌진한 택시… 1명 부상


가상화폐 아크 900원 상승한 3,125원에 거래 5315
가상화폐 아크 3,160원(+42.02%) 거래중
가상화폐 아크 3,385원(+52.13%) 거래중
가상화폐 비트코인 1,266,000원 상승한 67,217,000원에 거래
가상화폐 비트코인 1,080,000원 상승한 67,031,000원에 거래
가상화폐 비트코인 969,000원 상승한 66,920,000원에 거래
가상화폐 비트코인 734,000원 상승한 66,685,000원에 거래
가상화폐 아하토큰 7.3원 상승한 26.4원에 거래


민주노총, '청년에게 양질의 일자리를' 5318
민주노총, '청년에게 양질의 일자리를'
민주노총, '청년에게 양질의 일자리를'
민주노총, '청년에게 양질의 일자리를'


카카오뱅크, 신규 체크카드 디자인 공개…19일까지 고객 이벤트 진행 5339
카카오뱅크, 체크카드 신규 디자인 공개
카카오뱅크, 새 체크카드 디자인 공개


SK그룹 종합역량검사 끝마친 수험생들 5342
SK그룹 종합역량검사 마친 수험생들


김만배 내일 검찰 소환…로비 실체 드러날까 5344
김만배 내일 검찰 소환…로비 실체 드러날까


Ducks Kings Hockey 5369
Ducks Kings Hockey


이재명 "지사직 사퇴 고심…중도표 때문에 이승만 참배 안해" 5392
[단독] 이재명 "지사직 사퇴 고심…중도 잡자고 이승만 참배는 못해"


서울 떠나는 차량 행렬, 연휴 뒤 확산세 우려 5404
서울 떠나는 차량 행렬, 연휴 뒤 확산세 우려
서울 떠나는 차량 행렬, 연휴 뒤 확산세 우려
서울 떠나는 차

HUNGARY SWIMMING FINA WORLD CUP
HUNGARY SWIMMING FINA WORLD CUP


PORTUGAL LISBON FASHION WEEK 2021 8019
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021
PORTUGAL LISBON FASHION WEEK 2021


AUSTRIA GOVERNMENT KURZ RESIGNATION 8032
AUSTRIA GOVERNMENT KURZ RESIGNATION
epaselect AUSTRIA GOVERNMENT KURZ RESIGNATION
AUSTRIA GOVERNMENT KUR

"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수사 '급물살'
"상당히 진척"…공수처, 고발사주 수

[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘의 운세] 2021년 10월 10일 별자리 운세
[오늘의 운세] 2021년 10월 10일 띠별 운세
[오늘

In [39]:
multi_document_summary(725)


21 다만 자신의 정리된 마음은 정리되는 대로 여러분께 말씀드리겠다면서, 늘 차분한 마음과 책임있는 마음으로 기다려주기 바란다고 말했습니다.
16 [앵커]이낙연 전 대표 측에선 반발하고 있다고요?[기자]네, 이낙연 캠프 소속 의원 전원은 오늘 경선이 끝난 뒤 긴급회의를 열었습니다.
29 여유롭던 분위기가 달라진 건 30만 명 규모의 3차 선거인단 투표 결과가 발표된 때였습니다.


In [11]:
stand_idx = 586
for idx in sim_dict[stand_idx]:
    print(df['title'][idx])
print("*************")

content_list = []
print(df['content'][stand_idx])
content_list.append(df['content'][stand_idx])
print("************")
for idx in sim_dict[stand_idx]:
    print(df['content'][idx])
    content_list.append(df['content'][idx])
    print("************")

신용카드 캐시백 1천143만명 신청… 9일간 90억원 발생
정부 "신용카드 캐시백 9일간 90억원 발생"
신용카드 캐시백 9일간 90억원 발생…내달 15일 10월분 지급
신용카드 캐시백 9일간 90억원 발생…내달 15일 지급
*************
늘어난 카드 사용액의 10%를 월 최대 10만원까지 돌려주는 신용카드 캐시백(상생소비지원금)이 이달 들어 100억원 가까이 발생한 것으로 나타났다.기획재정부는 다음달 15일 지급될 예정인 10월분 캐시백 지급액이 지난 1~9일 현재 90억원으로 집계됐다고 10일 밝혔다.기재부 관계자는 "참여자 수와 캐시백 발생 구조 등을 고려할 때 캐시백 지급 예정액은 이달 후반으로 갈수록 빠르게 증가할 전망"이라며 "재원 소진 시에는 사업이 조기 종료될 수 있다"고 말했다.캐시백 신청자는 이달 9일 현재 1천143만명으로 집계됐다. 이들 가운데 외국인은 11만명(1.0%)이었다.신청 대비 확정 비율은 99%에 달해 신청자 대부분이 지원 대상 요건을 충족한 것으로 나타났다.카드 캐시백은 월간 카드 사용액이 2분기 월평균 사용액보다 3% 이상 증가하면 초과분의 10%를 캐시백(현금성 충전금)으로 환급해주는 제도다.예를 들어 2분기 월평균 사용액이 100만원이고 10월 카드 사용액이 153만원인 경우, 증가액 53만원 중 3만원(3%)을 제외한 50만원의 10%인 5만원을 캐시백으로 돌려준다.1인당 최대 지급액은 월별 10만원이다.캐시백은 사업 종료 전까지 언제든 신청할 수 있다. 만 19세 이상(2002년 12월 31일 이전 출생자)이고, 2분기 본인 명의 신용·체크카드 사용 실적이 있는 사람이 대상이다.신청 희망자는 롯데·비씨·삼성·신한·우리·KB국민·NH농협 등 9개 카드사 중 하나를 전담 카드사로 지정해 해당 카드사에 신청하면 된다.카드 사용액은 신청 일자와 상관없이 10월 1일 사용분부터 인정되며, 산정 기준이 되는 2분기 카드 사용 실적은 카드사 지정 신청 후 2일 이내에 확인할 수 있다.카드 사용처는 대부분 인정되나 대형

In [12]:
def content_refine_temp(content):
    content = re.sub("\([^\)]*\)","", content)
    content = re.sub("\<[^\]]*\>","", content)
    content = re.sub("[\n\t]+",". ", content)
    content = re.sub("([^\s]*\s기자)","", content)
    content = re.sub("([\w]*@[^\s]*)", "", content)
    content = re.sub("\〈[^〉]*\〉","", content)
    content = re.sub("(\s=\s)","", content)
    content = re.sub("(\s{2,})"," ", content)
    return content

In [13]:
content_refine_temp(content_list[2])

'동영상 뉴스. 정부가 소비 진작책으로 내놓은 소비 신용카드 캐시백, 상생소비지원금이 이달 들어 100억 원 가까이 발생한 것으로 나타났습니다.기획재정부는 다음 달 15일 지급될 예정인 10월분 캐시백 지급액이 이달 들어 9일까지 90억 원으로 집계됐다고 밝혔습니다.카드 캐시백은 월간 카드 사용액이 2분기 월평균 사용액보다 3% 이상 증가하면 초과분의 10%를 캐시백으로 환급해주는 제도입니다.이 제도는 10월과 11월, 두 달 동안 시행되고 1인당 캐시백 한도는 월 10만 원씩 최대 20만 원입니다.연합뉴스TV 기사문의 및 제보 : 카톡/라인 jebo23'

In [14]:
for i in range(len(content_list)):
    content_list[i] = content_refine_temp(content_list[i])
content_list

['늘어난 카드 사용액의 10%를 월 최대 10만원까지 돌려주는 신용카드 캐시백이 이달 들어 100억원 가까이 발생한 것으로 나타났다.기획재정부는 다음달 15일 지급될 예정인 10월분 캐시백 지급액이 지난 1~9일 현재 90억원으로 집계됐다고 10일 밝혔다.기재부 관계자는 "참여자 수와 캐시백 발생 구조 등을 고려할 때 캐시백 지급 예정액은 이달 후반으로 갈수록 빠르게 증가할 전망"이라며 "재원 소진 시에는 사업이 조기 종료될 수 있다"고 말했다.캐시백 신청자는 이달 9일 현재 1천143만명으로 집계됐다. 이들 가운데 외국인은 11만명이었다.신청 대비 확정 비율은 99%에 달해 신청자 대부분이 지원 대상 요건을 충족한 것으로 나타났다.카드 캐시백은 월간 카드 사용액이 2분기 월평균 사용액보다 3% 이상 증가하면 초과분의 10%를 캐시백으로 환급해주는 제도다.예를 들어 2분기 월평균 사용액이 100만원이고 10월 카드 사용액이 153만원인 경우, 증가액 53만원 중 3만원을 제외한 50만원의 10%인 5만원을 캐시백으로 돌려준다.1인당 최대 지급액은 월별 10만원이다.캐시백은 사업 종료 전까지 언제든 신청할 수 있다. 만 19세 이상이고, 2분기 본인 명의 신용·체크카드 사용 실적이 있는 사람이 대상이다.신청 희망자는 롯데·비씨·삼성·신한·우리·KB국민·NH농협 등 9개 카드사 중 하나를 전담 카드사로 지정해 해당 카드사에 신청하면 된다.카드 사용액은 신청 일자와 상관없이 10월 1일 사용분부터 인정되며, 산정 기준이 되는 2분기 카드 사용 실적은 카드사 지정 신청 후 2일 이내에 확인할 수 있다.카드 사용처는 대부분 인정되나 대형마트, 대형 백화점, 대형 종합온라인몰, 대형 전자판매점이나 명품전문매장, 유흥업 사용액 등은 대상에서 배제된다.연회비나 세금, 보험료 등 비소비성 지출도 대상에 포함되지 않는다.',
 '기재부 "1~9일 현재 90억원 집계" 신청자 99% 캐시백 대상 확정늘어난 카드 사용액의 10%를 월 최대 10만원까지 돌려주는 신용카드 캐시백이 

In [15]:
sentence_list = []
for content in content_list:
    for sentence in content.split("다."):
        if sentence == "":
            continue
        else:  
            sentence_list.append(sentence+"다.")
sentence_list

['늘어난 카드 사용액의 10%를 월 최대 10만원까지 돌려주는 신용카드 캐시백이 이달 들어 100억원 가까이 발생한 것으로 나타났다.',
 '기획재정부는 다음달 15일 지급될 예정인 10월분 캐시백 지급액이 지난 1~9일 현재 90억원으로 집계됐다고 10일 밝혔다.',
 '기재부 관계자는 "참여자 수와 캐시백 발생 구조 등을 고려할 때 캐시백 지급 예정액은 이달 후반으로 갈수록 빠르게 증가할 전망"이라며 "재원 소진 시에는 사업이 조기 종료될 수 있다"고 말했다.',
 '캐시백 신청자는 이달 9일 현재 1천143만명으로 집계됐다.',
 ' 이들 가운데 외국인은 11만명이었다.',
 '신청 대비 확정 비율은 99%에 달해 신청자 대부분이 지원 대상 요건을 충족한 것으로 나타났다.',
 '카드 캐시백은 월간 카드 사용액이 2분기 월평균 사용액보다 3% 이상 증가하면 초과분의 10%를 캐시백으로 환급해주는 제도다.',
 '예를 들어 2분기 월평균 사용액이 100만원이고 10월 카드 사용액이 153만원인 경우, 증가액 53만원 중 3만원을 제외한 50만원의 10%인 5만원을 캐시백으로 돌려준다.',
 '1인당 최대 지급액은 월별 10만원이다.',
 '캐시백은 사업 종료 전까지 언제든 신청할 수 있다.',
 ' 만 19세 이상이고, 2분기 본인 명의 신용·체크카드 사용 실적이 있는 사람이 대상이다.',
 '신청 희망자는 롯데·비씨·삼성·신한·우리·KB국민·NH농협 등 9개 카드사 중 하나를 전담 카드사로 지정해 해당 카드사에 신청하면 된다.',
 '카드 사용액은 신청 일자와 상관없이 10월 1일 사용분부터 인정되며, 산정 기준이 되는 2분기 카드 사용 실적은 카드사 지정 신청 후 2일 이내에 확인할 수 있다.',
 '카드 사용처는 대부분 인정되나 대형마트, 대형 백화점, 대형 종합온라인몰, 대형 전자판매점이나 명품전문매장, 유흥업 사용액 등은 대상에서 배제된다.',
 '연회비나 세금, 보험료 등 비소비성 지출도 대상에 포함되지 않는다.',
 '기재부 "1~9일 현재 

In [16]:
vocab_dict = {}
idx = 0
for sentence in sentence_list:
    for term in sentence.split():
        if term not in vocab_dict:
            vocab_dict[term] = idx
            idx+=1
            
terms_list = []
matrix = []
for sentence in sentence_list:
    terms = set(sentence.split())
    vec = [0.0] * len(vocab_dict)
    for term in terms:
        vec[vocab_dict[term]] = 1
    matrix.append(vec)
    terms_list.append(sentence.split())

In [17]:
def cosine_similarity(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(a))

In [18]:
sim_matrix = []
for i in tqdm(range(len(matrix))):
    sim_vec = []
    for j in range(len(matrix)):
        score = cosine_similarity(matrix[i], matrix[j])
        sim_vec.append(score)
    sim_matrix.append(sim_vec)

In [20]:
pd.DataFrame(sim_matrix)

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,1.000000,0.000000,0.058824,0.058824,0.0,0.117647,0.117647,0.117647,0.058824,0.000000,...,0.117647,0.117647,0.058824,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.0
1,0.000000,1.000000,0.066667,0.133333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.037037,0.037037,1.000000,0.074074,0.0,0.000000,0.000000,0.000000,0.000000,0.037037,...,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.037037,0.000000,0.000000,0.0
3,0.142857,0.285714,0.285714,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,0.000000,0.000000,0.000000,0.000000,0.0,0.066667,0.000000,0.066667,0.000000,0.000000,...,0.000000,0.066667,0.000000,0.000000,0.000000,1.000000,0.133333,0.000000,0.066667,0.0
58,0.043478,0.000000,0.043478,0.000000,0.0,0.043478,0.086957,0.130435,0.000000,0.086957,...,0.086957,0.130435,0.000000,0.086957,0.086957,0.086957,1.000000,0.043478,0.000000,0.0
59,0.066667,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,0.066667,0.000000,0.000000,...,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,1.000000,0.000000,0.0
60,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,1.000000,0.0


In [21]:
#representive_scoring
def rep_score_return(sim_vec, idx):
    count = 0
    for i in range(len(sim_vec)):
        if i == idx:
            continue
        else:
            if sim_vec[i] > 0:
                count += 1
    return count / len(sim_vec)

In [22]:
#length_scoring
def len_score_return(terms, max_el, max_rl):
    if max_rl / len(terms) == 1:
        return len(set(terms))/max_el * math.log10(1)
    else:
        return len(set(terms))/max_el * math.log10(max_rl/len(terms))

In [23]:
#diversity_scoring
def div_score_return(sim_vec, idx):
    min_score = 99
    for i, score in enumerate(sim_vec):
        if i!=idx and score > 0 and score < min_score:
            min_score = score
    return min_score

In [24]:
max_el = 0
max_rl = 0
for term in terms_list:
    if len(term) > max_rl:
        max_rl = len(term)
    
    if len(set(term)) > max_el:
        max_el = len(set(term))

score_list = []
for i in range(len(matrix)):
    sim_vec = matrix[i]
    idx = i
    
    rep_score = rep_score_return(sim_vec, idx)
    len_score = len_score_return(terms_list[idx], max_el, max_rl)
    div_score = div_score_return(sim_vec, idx)
    
    score = rep_score * len_score * div_score
#     score = math.log10(rep_score) + math.log10(len_score) + math.log10(div_score)
    score_list.append(score)

In [25]:
score_list

[0.008878171041839819,
 0.008797316044967985,
 0.005023304342530136,
 0.003842983005819487,
 0.0017169011159542653,
 0.008361252466464698,
 0.008047973981035619,
 0.008694424124809254,
 0.0023947836679654444,
 0.004578402975878041,
 0.007288438384886939,
 0.008797316044967985,
 0.006738839584567772,
 0.007344075310345439,
 0.005302222549824985,
 0.007511763393987614,
 0.008797316044967985,
 0.003842983005819487,
 0.008361252466464698,
 0.008047973981035619,
 0.008715830117268649,
 0.007855741420652174,
 0.007288438384886939,
 0.008797316044967985,
 0.005302222549824985,
 0.009156805951756082,
 0.007344075310345439,
 0.005302222549824985,
 0.005023304342530136,
 0.008580630912499687,
 0.009156805951756082,
 0.008047973981035619,
 0.009433056731954808,
 0.003842983005819487,
 0.008797316044967985,
 0.008797316044967985,
 0.0031097854331874337,
 0.0031097854331874337,
 0.008361252466464698,
 0.008248136510731884,
 0.004578402975878041,
 0.007855741420652174,
 0.006667613174021316,
 0.0045

In [26]:
temp_idx = np.array(score_list).argsort()[::-1][:2]
temp_idx = sorted(temp_idx)
for sen_idx in temp_idx:
    print(sen_idx,sentence_list[sen_idx])
    print()

30 기획재정부는 다음 달 15일 지급될 예정인 10월분 캐시백 지급액이 이달 들어 9일까지 90억 원으로 집계됐다고 밝혔습니다.

32 이 제도는 10월과 11월, 두 달 동안 시행되고 1인당 캐시백 한도는 월 10만 원씩 최대 20만 원입니다.



In [27]:
def multi_document_summary(stand_idx):
    content_list = []
    content_list.append(df['content'][stand_idx])
    for idx in sim_dict[stand_idx]:
        content_list.append(df['content'][idx])
    for i in range(len(content_list)):
        content_list[i] = content_refine_temp(content_list[i])
    sentence_list = []
    for content in content_list:
        for sentence in content.split("다."):
            if sentence == "":
                continue
            else:  
                sentence_list.append(sentence+"다.")
    vocab_dict = {}
    idx = 0
    for sentence in sentence_list:
        for term in sentence.split():
            if term not in vocab_dict:
                vocab_dict[term] = idx
                idx+=1

    terms_list = []
    matrix = []
    for sentence in sentence_list:
        terms = set(sentence.split())
        vec = [0.0] * len(vocab_dict)
        for term in terms:
            vec[vocab_dict[term]] = 1
        matrix.append(vec)
        terms_list.append(sentence.split())
    sim_matrix = []
    for i in tqdm(range(len(matrix))):
        sim_vec = []
        for j in range(len(matrix)):
            score = cosine_similarity(matrix[i], matrix[j])
            sim_vec.append(score)
        sim_matrix.append(sim_vec)
    max_el = 0
    max_rl = 0
    for term in terms_list:
        if len(term) > max_rl:
            max_rl = len(term)

        if len(set(term)) > max_el:
            max_el = len(set(term))

    score_list = []
    for i in range(len(matrix)):
        sim_vec = matrix[i]
        idx = i

        rep_score = rep_score_return(sim_vec, idx)
        len_score = len_score_return(terms_list[idx], max_el, max_rl)
        div_score = div_score_return(sim_vec, idx)

        score = rep_score * len_score * div_score
        score_list.append(score)
    temp_idx = np.array(score_list).argsort()[::-1][:3]
#     temp_idx = sorted(temp_idx)
    for sen_idx in temp_idx:
        print(sen_idx,sentence_list[sen_idx])